# Más `pandas`

## Manejando datos faltantes

Hemos visto que `pandas` automágicamente es capaz de manejar valores faltantes o inexistentes, a través de distintas etiquetas como `NaN`, `NA`, etc., dependiendo del tipo de dato que se esté utilizando. Más allá de la lectura de los datos, muchos de los métodos nativos de `pandas` son capaces de trabajar aún cuando falten datos. Sin embargo, puede ser necesario intervenir efectivamente sobre esos datos para poder continuar con ese procesamiento.

In [ ]:
import numpy as np 
import pandas as pd

In [ ]:
d = np.array([['auto','moto',np.nan,None,'bicicleta'],[4,2,0.0,None,2]])
d


In [ ]:
s = pd.DataFrame(d.T,columns=['vehículos','ruedas'])
s

In [ ]:
s.isna()


In [ ]:
s.dropna() # equivalente a s[s.notna()]

In [ ]:
s.dropna(axis=1) # equivalente a s.dropna(axis='columns')

In [ ]:
s.dropna(how='all') # elimina las filas que tengan todos los valores nulos

También se puede usar el argumento 'thresh=<valor>' para acotar la cantidad de valores inexistentes que se quieren eliminar. Por ejemplo, `thresh=3` eliminará todas aquellas filas que tienen 3 o más valores faltantes.

Es posible también que uno no pueda trabajar con valores inexistentes, y tiene que cambiarlos por algún valor. Para ello está el método `fillna`.

In [ ]:
df = pd.DataFrame(np.random.standard_normal((7, 3)),columns=['A','B','C'])

df.iloc[:4,2] = np.nan
df.iloc[1:3,0] = None 

df

In [ ]:
df.fillna(0) # rellena los valores nulos con 0

In [ ]:
df.fillna({'A':0,'C':2}) # rellena los valores nulos con 0, 1 y 2 respectivamente

In [ ]:
df.ffill() # rellena los valores nulos con el valor anterior

In [ ]:
df.ffill(axis=1) # rellena los valores nulos con el valor anterior en la misma fila

In [ ]:
df.bfill() # rellena los valores nulos con el valor siguiente

Estos métodos para reemplazar de valores inexistentes son un caso particular de un método para reemplazar valores en forma general, denominado `replace` y puede ser útil para reemplazar valores que, por alguna razón, se encuentran fuera del rango esperado de los datos (un precio negativo, una edad mayor a 120 años, etc.). Veamos cómo funciona:

In [ ]:
p = pd.Series([23,4,-8,12,27,-9])
p

In [ ]:
p.replace(-9,np.nan) # reemplaza -9 por NaN

In [ ]:
p.replace({-9:np.nan,-8:0}) # reemplaza -9 por NaN y 23 por 0

In [ ]:
p<0 

In [ ]:
p[p < 0]

In [ ]:
list(p[p < 0])

In [ ]:
p.replace(list(p[p < 0]),[86,22]) # encuentro los valores < 0 y los reemplazo por 86 y 22 respectivamente

> Notar que el método `replace` genera un nuevo dato.

## Indicadores

Otro tipo de transformación para el modelado estadístico es convertir una variable en un
_indicador_. Si una columna en un `DataFrame` tiene k valores distintos, se derivará una matriz
o `DataFrame` con k columnas conteniendo unos y ceros, por ejemplo:

In [ ]:
df = pd.DataFrame({'key': ['a','a','b','d','a','c','c'],'datos': np.random.standard_normal(7)})
df

In [ ]:
pd.get_dummies(df['key'],dtype=int) # crea variables dummy

## Manejando índices múltiples

Hemos visto hasta ahora que los índices nos etiquetan cada una de las filas de un `DataFrame`. Pandas tiene la posibilidad de utilizar índices múltiples o _jerárquicos_ con el objeto de añadir dimensionalidad a las tablas. 
La implementación de esta característica consiste en utilizar tuplas como índices para etiquetar cada fila:


In [ ]:
# Índices jerárquicos: ciudades, productos, años
index = [
    ("Buenos Aires", "Zapatos", 2022),
    ("Buenos Aires", "Ropa", 2022),
    ("Buenos Aires", "Ropa", 2023),
    ("Córdoba", "Zapatos", 2023),
    ("Córdoba", "Ropa", 2023),
    ("Rosario", "Zapatos", 2023),
]

In [ ]:
index = pd.MultiIndex.from_tuples(index, names=["Ciudad", "Producto", "Año"])
print(type(index))
index

In [ ]:
# Datos
data = {
    "Ventas": [200, 150, 300, 400, 250, 500],
    "Costo": [120, 80, 180, 240, 150, 300]
}

In [ ]:
df = pd.DataFrame(data, index=index)
df

Si queremos obtener ciertas filas específicas, usamos `.loc`. 

In [ ]:
# Acceso por niveles del índice
print("\nDatos de 'Buenos Aires' en 2022:")
print(df.loc[("Buenos Aires", slice(None), 2022), :])

La función `slice` se usa para determinar el rango de filas en cada componente del índice. `slice(None)` implica usar todos los valores posibles para dicha componente del índice.

### Agrupando

La potencia de los índices múltiples radica en poder agrupar datos de acuerdo a una determinada componente del índice. Para ello se utiliza el método `.groupby()`, que agrupa los valores de acuerdo al nivel (`level`) indicado: 

In [ ]:
# Resumen por nivel del índice
print("\nVentas totales por ciudad:")
print(df.groupby(level="Ciudad")["Ventas"].sum())

Si uno quisiera calcular el monto total de ventas por ciudad y por año, por ejemplo, se podría hacer:

In [ ]:
# Agrupar por 'Ciudad' y 'Año' y sumar el costo
df["Total"] = df["Ventas"] * df["Costo"]
df


In [ ]:
costo_anual_ciudad_x_año = df.groupby(level=["Ciudad", "Año"])["Total"].sum()

# Mostrar el resultado
print("Costo anual por ciudad por año:")
print(costo_anual_ciudad_x_año)


In [ ]:
costo_anual_ciudad = df.groupby(level=["Ciudad"])["Total"].sum()

# Mostrar el resultado
print("Costo anual por ciudad:")
print(costo_anual_ciudad)

## Apilando y desapilando

Otra operación es apilar o desapilar el dataframe de índices múltiples:

In [ ]:
df.unstack() # desapila el índice

In [ ]:
df.unstack().columns

Tal como se ve en el ejemplo anterior, **las columnas también pueden ser descriptas con índices jerárquicos**

In [ ]:
df.unstack().unstack() # desapila el índice

## Categorías

Es muy posible que en nuestro conjunto de datos tengamos valores repetidos. 


In [ ]:
s = pd.Series(["Pequeño", "Mediano", "Grande", "Pequeño", "Grande", "Mediano"])
s

In [ ]:
s_cat = s.astype("category")
s_cat

In [ ]:
s_cat.cat.categories


In [ ]:
s_cat.cat.codes

In [ ]:
dict(enumerate(s_cat.cat.categories))

En el caso de un `DataFrame`, uno puede convertir una columna en una categoría reasignandola:

In [ ]:
precios = [100, 200, 300, 150, 250, 180]

df = pd.DataFrame({'precios':precios,'tamaño':s})
df

In [ ]:
df['tamaño']

In [ ]:
df['tamaño'] = df['tamaño'].astype('category')
df['tamaño']

El beneficio principal del uso de categorías tiene que ver con la eficiencia en la memoria y en las operaciones:

In [ ]:
N = 10_000_000

s = pd.Series(['a','b','c','d']* (N//4))
s_cat = s.astype('category')

In [ ]:
print(s.memory_usage(deep=True))
print(s_cat.memory_usage(deep=True))

In [ ]:
%timeit s.value_counts()

In [ ]:
%timeit s_cat.value_counts()

-----

## Ejercicio 15(a)

1. Retome el `DataFrame` creado en el ejercicio 14(a), y genere un nuevo `DataFrame` utilizando adecuadamente índices jerárquicos.

-----